In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [276]:
import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT IS NOT NULL AND (CATEGORY_EVENT='Long' OR CATEGORY_EVENT='Sprint' OR CATEGORY_EVENT='Mid' OR CATEGORY_EVENT='Hurdles' OR CATEGORY_EVENT='Relay' OR CATEGORY_EVENT='Walk' OR CATEGORY_EVENT='Marathon' OR CATEGORY_EVENT='Steeple')
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [277]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,Supanich Poolkerd,11.63,None,1,100m,None,Sprint,Female,SEA Games,None
1,Kristina Knott,11.66,None,2,100m,None,Sprint,Female,SEA Games,None
2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,None,Sprint,Female,SEA Games,None
3,Hoang Du Y,11.73,None,4,100m,None,Sprint,Female,SEA Games,None
4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,None,Sprint,Female,SEA Games,None
...,...,...,...,...,...,...,...,...,...,...
17303,"Aw, Hailey",37.68,11,28,Girls 11-12 200 Meter Dash U13,2012-09-08 00:00:00.000,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17304,"Wang, Minqi",29.97,12,4,Girls 11-12 200 Meter Dash U13,2011-03-18 00:00:00.000,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17305,"Koh, Jayna Jia Y",33.03,12,19,Girls 11-12 200 Meter Dash U13,2011-03-18 00:00:00.000,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17306,"Mak, Chen Yu, Adelyn",34.73,11,25,Girls 11-12 200 Meter Dash U13,2012-03-06 00:00:00.000,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [256]:
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500m Race Walk.+', value='1500m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Shot Put OPEN', value='Shot put')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Women Shot Put U18', value='Shot put')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Discus Throw U18', value='Discus throw')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Men Discus Throw OPEN', value='Discus throw')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw Open', value='Javelin throw')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw Open', value='Javelin throw')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump Open.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U15.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U18.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump Men.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump U20.+', value='Long jump')




athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Novice.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Intermediate.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Open.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U17.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U15.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump U13.+', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump Masters.+', value='High jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase (0.914m).+', value='3000m steeplechase (0.914m)')




In [257]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,Supanich Poolkerd,11.63,None,1,100m,Sprint,Female,SEA Games,None
1,Kristina Knott,11.66,None,2,100m,Sprint,Female,SEA Games,None
2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,Sprint,Female,SEA Games,None
3,Hoang Du Y,11.73,None,4,100m,Sprint,Female,SEA Games,None
4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,Sprint,Female,SEA Games,None
...,...,...,...,...,...,...,...,...,...
17303,"Aw, Hailey",37.68,11,28,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17304,"Wang, Minqi",29.97,12,4,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17305,"Koh, Jayna Jia Y",33.03,12,19,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
17306,"Mak, Chen Yu, Adelyn",34.73,11,25,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [258]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3' AND (CATEGORY_EVENT='Sprint' OR CATEGORY_EVENT='Mid' OR CATEGORY_EVENT='Hurdles' OR CATEGORY_EVENT='Relay' OR CATEGORY_EVENT='Walk' OR CATEGORY_EVENT='Marathon' OR CATEGORY_EVENT='Steeple')
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [259]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final
1,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final
2,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final
3,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final
4,Goh Chui Ling,4:26.33,3,1500m,Mid,Female,SEA Games,Final
5,Odekta Elvina Naibaho,35:31.03,3,10000m,Sprint,Female,SEA Games,Final
6,Dina Aulia,13.59,3,100m hurdles,Hurdles,Female,SEA Games,Final
7,Nguyễn Thị Ngọc,59.09,3,400m hurdles,Hurdles,Female,SEA Games,Final
8,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final
9,"Azreen Nabila Alias, Nur Afrina Batrisyia, N...",44.58,3,4 x 100m relay,Relay,Female,SEA Games,Final


In [260]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [261]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [262]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,11.75,100m,Female
1,23.6,200m,Female
2,53.84,400m,Female
3,2:09.15,800m,Female
4,4:26.33,1500m,Female
5,35:31.03,10000m,Female
6,13.59,100m hurdles,Female
7,59.09,400m hurdles,Female
8,11:00.85,3000m steeplechase,Female
9,44.58,4 x 100m relay,Female


In [263]:
def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))

        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [264]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    metric=benchmarks.iloc[rowIndex,0]
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [265]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,11.75,100m,Female,11.750
1,23.6,200m,Female,23.600
2,53.84,400m,Female,53.840
3,2:09.15,800m,Female,129.150
4,4:26.33,1500m,Female,266.330
5,35:31.03,10000m,Female,2131.030
6,13.59,100m hurdles,Female,13.590
7,59.09,400m hurdles,Female,59.090
8,11:00.85,3000m steeplechase,Female,660.850
9,44.58,4 x 100m relay,Female,44.580


In [266]:
benchmarks['2pc']=benchmarks['Metric']*1.02
benchmarks['35pc']=benchmarks['Metric']*1.035
benchmarks['5pc']=benchmarks['Metric']*1.05

In [267]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,11.75,100m,Female,11.750,11.98500,12.161250,12.33750
1,23.6,200m,Female,23.600,24.07200,24.426000,24.78000
2,53.84,400m,Female,53.840,54.91680,55.724400,56.53200
3,2:09.15,800m,Female,129.150,131.73300,133.670250,135.60750
4,4:26.33,1500m,Female,266.330,271.65660,275.651550,279.64650
5,35:31.03,10000m,Female,2131.030,2173.65060,2205.616050,2237.58150
6,13.59,100m hurdles,Female,13.590,13.86180,14.065650,14.26950
7,59.09,400m hurdles,Female,59.090,60.27180,61.158150,62.04450
8,11:00.85,3000m steeplechase,Female,660.850,674.06700,683.979750,693.89250
9,44.58,4 x 100m relay,Female,44.580,45.47160,46.140300,46.80900


In [268]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['EVENT','GENDER'], how='left')


In [269]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
0,0,Supanich Poolkerd,11.63,None,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
1,1,Kristina Knott,11.66,None,2,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
2,2,Zaidatul Husniah Zulkifli,11.69,None,3,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
3,3,Hoang Du Y,11.73,None,4,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
4,4,Elizabeth-Ann Tan Shee Ru,12.0,None,5,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17303,17303,"Aw, Hailey",37.68,11,28,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17304,17304,"Wang, Minqi",29.97,12,4,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17305,17305,"Koh, Jayna Jia Y",33.03,12,19,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800
17306,17306,"Mak, Chen Yu, Adelyn",34.73,11,25,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800


In [270]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'—', value=np.NaN)


In [271]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'–', value=np.NaN)


In [272]:
for i in range(len(df)):
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,5]
    
    
    metric=df.iloc[rowIndex,2]
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS':
        continue
    
        
    out = convert_time(i, input_string, metric)
         
    df.loc[rowIndex, 'RESULT_CONV'] = out

100m
11.63
100m 11.63 11.63
100m
11.66
100m 11.66 11.66
100m
11.69
100m 11.69 11.69
100m
11.73
100m 11.73 11.73
100m
12.0
100m 12.0 12.0
100m
12.39
100m 12.39 12.39
100m
11.49
100m 11.49 11.49
100m
11.71
100m 11.71 11.71
100m
12.04
100m 12.04 12.04
100m
12.28
100m 12.28 12.28
100m
12.4
100m 12.4 12.4
100m
12.59
100m 12.59 12.59
100m
12.71
100m 12.71 12.71
100m
11.41
100m 11.41 11.41
100m
11.58
100m 11.58 11.58
100m
11.75
100m 11.75 11.75
100m
11.83
100m 11.83 11.83
100m
11.85
100m 11.85 11.85
100m
11.96
100m 11.96 11.96
100m
12.12
100m 12.12 12.12
200m
24.03
200m 24.03 24.03
200m
24.3
200m 24.3 24.3
200m
24.4
200m 24.4 24.4
200m
25.73
200m 25.73 25.73
200m
25.73
200m 25.73 25.73
200m
23.98
200m 23.98 23.98
200m
24.06
200m 24.06 24.06
200m
24.12
200m 24.12 24.12
200m
24.15
200m 24.15 24.15
200m
25.5
200m 25.5 25.5
200m
25.96
200m 25.96 25.96
200m
22.69
200m 22.69 22.69
200m
23.54
200m 23.54 23.54
200m
23.6
200m 23.6 23.6
200m
23.79
200m 23.79 23.79
200m
24.03
200m 24.03 24.03
200m
24.09

400m
1:06.72
400m 1:06.72 66.72
400m
1:13.23
400m 1:13.23 73.23
400m
1:29.49
400m 1:29.49 89.49
400m
1:15.31
400m 1:15.31 75.31
400m
1:06.99
400m 1:06.99 66.99
400m
1:10.01
400m 1:10.01 70.01
400m
1:17.59
400m 1:17.59 77.59
Women 100 Meter Hurdles (0.762m) U18
19.22
Women 100 Meter Hurdles (0.762m) U18 19.22 19.22
Women 100 Meter Hurdles (0.762m) U18
15.99
Women 100 Meter Hurdles (0.762m) U18 15.99 15.99
Women 100 Meter Hurdles (0.762m) U18
18.27
Women 100 Meter Hurdles (0.762m) U18 18.27 18.27
Women 100 Meter Hurdles (0.762m) U18
18.79
Women 100 Meter Hurdles (0.762m) U18 18.79 18.79
Women 100 Meter Hurdles (0.762m) U18
18.65
Women 100 Meter Hurdles (0.762m) U18 18.65 18.65
Women 100 Meter Hurdles (0.762m) U18
19.78
Women 100 Meter Hurdles (0.762m) U18 19.78 19.78
100m
14.59
100m 14.59 14.59
100m
13.66
100m 13.66 13.66
100m
14.19
100m 14.19 14.19
100m
13.51
100m 13.51 13.51
100m
16.56
100m 16.56 16.56
100m
16.67
100m 16.67 16.67
100m
13.78
100m 13.78 13.78
400m
1:09.45
400m 1:09.45 69

200m
25.23
200m 25.23 25.23
200m
23.40
200m 23.40 23.4
200m
24.24
200m 24.24 24.24
400m
59.49
400m 59.49 59.49
400m
54.66
400m 54.66 54.66
400m
54.71
400m 54.71 54.71
400m
56.51
400m 56.51 56.51
Men 110 Meter Hurdles (0.991m) U20
18.29
Men 110 Meter Hurdles (0.991m) U20 18.29 18.29
Men 110 Meter Hurdles (0.991m) U20
23.14
Men 110 Meter Hurdles (0.991m) U20 23.14 23.14
Men 110 Meter Hurdles (0.991m) U20
18.03
Men 110 Meter Hurdles (0.991m) U20 18.03 18.03
Men 110 Meter Hurdles (0.991m) U20
14.55
Men 110 Meter Hurdles (0.991m) U20 14.55 14.55
Men 400 Meter Hurdles (0.914m) U20
1:03.01
Men 400 Meter Hurdles (0.914m) U20 1:03.01 63.01
Men 400 Meter Hurdles (0.914m) U20
1:04.45
Men 400 Meter Hurdles (0.914m) U20 1:04.45 64.45
Men 400 Meter Hurdles (0.914m) U20
1:01.62
Men 400 Meter Hurdles (0.914m) U20 1:01.62 61.62
100m
11.01
100m 11.01 11.01
100m
10.72
100m 10.72 10.72
100m
10.94
100m 10.94 10.94
100m
10.65
100m 10.65 10.65
200m
21.37
200m 21.37 21.37
200m
21.12
200m 21.12 21.12
200m
21.3

Boys 7-10 80 Meter Dash U11
12.20
Boys 7-10 80 Meter Dash U11 12.20 12.2
Boys 7-10 80 Meter Dash U11
11.60
Boys 7-10 80 Meter Dash U11 11.60 11.6
Boys 7-10 80 Meter Dash U11
13.75
Boys 7-10 80 Meter Dash U11 13.75 13.75
Women 16-99 1 Mile Run Open
5:56.77
Women 16-99 1 Mile Run Open 5:56.77 356.77
Women 16-99 1 Mile Run Open
5:56.77
Women 16-99 1 Mile Run Open 5:56.77 356.77
Women 16-99 1 Mile Run Open
5:56.77
Women 16-99 1 Mile Run Open 5:56.77 356.77
Women 16-99 1 Mile Run Open
7:22.43
Women 16-99 1 Mile Run Open 7:22.43 442.43
Women 16-99 1 Mile Run Open
7:22.43
Women 16-99 1 Mile Run Open 7:22.43 442.43
Women 16-99 1 Mile Run Open
7:22.43
Women 16-99 1 Mile Run Open 7:22.43 442.43
Women 16-99 1 Mile Run Open
6:07.54
Women 16-99 1 Mile Run Open 6:07.54 367.54
Women 16-99 1 Mile Run Open
6:07.54
Women 16-99 1 Mile Run Open 6:07.54 367.54
Women 16-99 1 Mile Run Open
6:07.54
Women 16-99 1 Mile Run Open 6:07.54 367.54
Women 16-99 1 Mile Run Open
6:22.57
Women 16-99 1 Mile Run Open 6:22.

1500m run
5:40.32
1500m run 5:40.32 340.32
800m
2:47.69
800m 2:47.69 167.69
400m
1:02.22
400m 1:02.22 62.22
800m
2:34.38
800m 2:34.38 154.38
100m
13.31
100m 13.31 13.31
100m
11.46
100m 11.46 11.46
100m
11.68
100m 11.68 11.68
400m
54.34
400m 54.34 54.34
200m
25.93
200m 25.93 25.93
100m
11.98
100m 11.98 11.98
Boys 3000 Meter Race Walk B Div
18:13.02
Boys 3000 Meter Race Walk B Div 18:13.02 1093.02
100m
16.16
100m 16.16 16.16
100m
12.40
100m 12.40 12.4
200m
25.69
200m 25.69 25.69
100m
13.13
100m 13.13 13.13
200m
27.77
200m 27.77 27.77
400m
1:08.92
400m 1:08.92 68.92
800m
2:37.33
800m 2:37.33 157.33
800m
2:55.82
800m 2:55.82 175.82
800m
3:01.71
800m 3:01.71 181.71
800m
2:55.60
800m 2:55.60 175.6
200m
27.59
200m 27.59 27.59
100m
16.02
100m 16.02 16.02
800m
3:30.05
800m 3:30.05 210.05
800m
3:05.19
800m 3:05.19 185.19
800m
3:30.01
800m 3:30.01 210.01
400m
1:03.53
400m 1:03.53 63.53
800m
2:37.17
800m 2:37.17 157.17
1500m run
5:36.35
1500m run 5:36.35 336.35
400m
1:01.81
400m 1:01.81 61.81
200m

...

01:00.0
400m 01:00.0 60.0
400m
00:56.8
400m 00:56.8 56.8
400m
01:06.7
400m 01:06.7 66.7
400m
01:07.9
400m 01:07.9 67.9
400m
00:55.7
400m 00:55.7 55.7
400m
00:58.6
400m 00:58.6 58.6
400m
01:00.4
400m 01:00.4 60.4
400m
01:06.3
400m 01:06.3 66.3
400m
01:05.5
400m 01:05.5 65.5
400m
01:06.2
400m 01:06.2 66.2
400m
00:58.1
400m 00:58.1 58.1
400m
01:07.8
400m 01:07.8 67.8
400m
00:58.2
400m 00:58.2 58.2
400m
01:05.3
400m 01:05.3 65.3
400m
00:59.3
400m 00:59.3 59.3
400m
01:14.7
400m 01:14.7 74.7
400m
00:57.1
400m 00:57.1 57.1
400m
01:03.3
400m 01:03.3 63.3
400m
00:56.4
400m 00:56.4 56.4
400m
01:05.0
400m 01:05.0 65.0
400m
01:29.3
400m 01:29.3 89.3
400m
00:55.3
400m 00:55.3 55.3
400m
00:59.6
400m 00:59.6 59.6
400m
01:01.4
400m 01:01.4 61.4
400m
01:17.0
400m 01:17.0 77.0
400m
01:00.0
400m 01:00.0 60.0
400m
00:55.0
400m 00:55.0 55.0
400m
00:56.8
400m 00:56.8 56.8
400m
00:51.4
400m 00:51.4 51.4
400m
00:56.0
400m 00:56.0 56.0
400m
00:52.7
400m 00:52.7 52.7
400m
00:55.1
400m 00:55.1 55.1
400m
00:53.0


800m
02:42.0
800m 02:42.0 162.0
800m
02:23.6
800m 02:23.6 143.6
800m
02:24.1
800m 02:24.1 144.1
800m
02:52.5
800m 02:52.5 172.5
800m
03:12.8
800m 03:12.8 192.8
800m
02:21.8
800m 02:21.8 141.8
800m
02:24.0
800m 02:24.0 144.0
800m
02:46.3
800m 02:46.3 166.3
800m
02:47.7
800m 02:47.7 167.7
800m
02:48.3
800m 02:48.3 168.3
800m
02:47.6
800m 02:47.6 167.6
800m
02:37.1
800m 02:37.1 157.1
800m
02:38.6
800m 02:38.6 158.6
800m
02:34.9
800m 02:34.9 154.9
800m
02:48.1
800m 02:48.1 168.1
800m
02:34.4
800m 02:34.4 154.4
800m
02:39.0
800m 02:39.0 159.0
800m
03:10.4
800m 03:10.4 190.4
800m
03:02.9
800m 03:02.9 182.9
800m
02:34.8
800m 02:34.8 154.8
800m
02:52.9
800m 02:52.9 172.9
800m
02:32.7
800m 02:32.7 152.7
800m
02:37.8
800m 02:37.8 157.8
800m
03:05.0
800m 03:05.0 185.0
800m
02:34.6
800m 02:34.6 154.6
800m
02:57.5
800m 02:57.5 177.5
800m
03:03.8
800m 03:03.8 183.8
800m
02:54.4
800m 02:54.4 174.4
800m
03:06.6
800m 03:06.6 186.6
800m
02:46.0
800m 02:46.0 166.0
800m
02:44.5
800m 02:44.5 164.5
800m
02:

4x100m Relay
00:56.2
4x100m Relay 00:56.2 56.2
4x100m Relay
00:55.1
4x100m Relay 00:55.1 55.1
4x100m Relay
00:54.0
4x100m Relay 00:54.0 54.0
4x100m Relay
00:56.1
4x100m Relay 00:56.1 56.1
4x100m Relay
01:01.4
4x100m Relay 01:01.4 61.4
4x100m Relay
01:02.5
4x100m Relay 01:02.5 62.5
4x100m Relay
00:54.3
4x100m Relay 00:54.3 54.3
4x100m Relay
00:56.2
4x100m Relay 00:56.2 56.2
4x100m Relay
00:55.6
4x100m Relay 00:55.6 55.6
4x100m Relay
00:55.1
4x100m Relay 00:55.1 55.1
4x100m Relay
00:55.1
4x100m Relay 00:55.1 55.1
4x100m Relay
00:53.8
4x100m Relay 00:53.8 53.8
4x100m Relay
00:53.3
4x100m Relay 00:53.3 53.3
4x100m Relay
00:54.8
4x100m Relay 00:54.8 54.8
4x100m Relay
00:53.2
4x100m Relay 00:53.2 53.2
4x100m Relay
01:02.8
4x100m Relay 01:02.8 62.8
4x100m Relay
01:00.7
4x100m Relay 01:00.7 60.7
4x100m Relay
01:02.9
4x100m Relay 01:02.9 62.9
4x100m Relay
00:56.9
4x100m Relay 00:56.9 56.9
4x100m Relay
01:02.4
4x100m Relay 01:02.4 62.4
4x100m Relay
01:00.2
4x100m Relay 01:00.2 60.2
4x100m Relay


400m
53.04
400m 53.04 53.04
400m
57.01
400m 57.01 57.01
400m
57.20
400m 57.20 57.2
400m
55.77
400m 55.77 55.77
400m
1:01.02
400m 1:01.02 61.02
400m
59.92
400m 59.92 59.92
400m
1:00.52
400m 1:00.52 60.52
400m
58.29
400m 58.29 58.29
400m
56.71
400m 56.71 56.71
400m
56.96
400m 56.96 56.96
400m
56.93
400m 56.93 56.93
400m
1:06.92
400m 1:06.92 66.92
400m
1:02.83
400m 1:02.83 62.83
400m
1:04.44
400m 1:04.44 64.44
400m
1:04.72
400m 1:04.72 64.72
400m
1:01.01
400m 1:01.01 61.01
400m
1:01.84
400m 1:01.84 61.84
400m
56.84
400m 56.84 56.84
400m
1:05.17
400m 1:05.17 65.17
400m
59.06
400m 59.06 59.06
400m
58.97
400m 58.97 58.97
400m
1:04.58
400m 1:04.58 64.58
400m
1:03.44
400m 1:03.44 63.44
400m
1:10.77
400m 1:10.77 70.77
400m
55.80
400m 55.80 55.8
400m
1:05.47
400m 1:05.47 65.47
400m
1:06.76
400m 1:06.76 66.76
400m
59.82
400m 59.82 59.82
400m
58.81
400m 58.81 58.81
400m
57.96
400m 57.96 57.96
400m
59.08
400m 59.08 59.08
400m
1:08.57
400m 1:08.57 68.57
400m
59.04
400m 59.04 59.04
400m
1:11.05
400m 

Girls 7-8 300 Meter Run U9
1:01.27
Girls 7-8 300 Meter Run U9 1:01.27 61.27
Girls 7-8 300 Meter Run U9
1:01.27
Girls 7-8 300 Meter Run U9 1:01.27 61.27
Girls 7-8 300 Meter Run U9
1:01.15
Girls 7-8 300 Meter Run U9 1:01.15 61.15
Girls 7-8 300 Meter Run U9
1:01.15
Girls 7-8 300 Meter Run U9 1:01.15 61.15
Girls 7-8 300 Meter Run U9
1:01.15
Girls 7-8 300 Meter Run U9 1:01.15 61.15
Girls 7-8 300 Meter Run U9
1:16.44
Girls 7-8 300 Meter Run U9 1:16.44 76.44
Girls 7-8 300 Meter Run U9
1:16.44
Girls 7-8 300 Meter Run U9 1:16.44 76.44
Girls 7-8 300 Meter Run U9
1:16.44
Girls 7-8 300 Meter Run U9 1:16.44 76.44
Girls 7-8 300 Meter Run U9
1:07.15
Girls 7-8 300 Meter Run U9 1:07.15 67.15
Girls 7-8 300 Meter Run U9
1:07.15
Girls 7-8 300 Meter Run U9 1:07.15 67.15
Girls 7-8 300 Meter Run U9
1:07.15
Girls 7-8 300 Meter Run U9 1:07.15 67.15
Girls 7-8 300 Meter Run U9
1:19.15
Girls 7-8 300 Meter Run U9 1:19.15 79.15
Girls 7-8 300 Meter Run U9
1:19.15
Girls 7-8 300 Meter Run U9 1:19.15 79.15
Girls 7-8 30

In [287]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,Supanich Poolkerd,11.63,0,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.63
1,1,Kristina Knott,11.66,0,2,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.66
2,2,Zaidatul Husniah Zulkifli,11.69,0,3,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.69
3,3,Hoang Du Y,11.73,0,4,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.73
4,4,Elizabeth-Ann Tan Shee Ru,12.0,0,5,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17303,17303,"Aw, Hailey",37.68,11,28,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800,37.68
17304,17304,"Wang, Minqi",29.97,12,4,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800,29.97
17305,17305,"Koh, Jayna Jia Y",33.03,12,19,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800,33.03
17306,17306,"Mak, Chen Yu, Adelyn",34.73,11,25,200m,Sprint,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,23.6,23.60,24.072,24.42600,24.7800,34.73


In [285]:
df["AGE"].fillna(0, inplace=True)


In [288]:
df['AGE'] = df['AGE'].astype('float')

In [309]:
rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [310]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,Supanich Poolkerd,11.63,0.0,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.63
1,1,Kristina Knott,11.66,0.0,2,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.66
2,2,Zaidatul Husniah Zulkifli,11.69,0.0,3,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.69
3,3,Hoang Du Y,11.73,0.0,4,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.73
6,6,Shanti Pereira,11.49,0.0,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.985,12.16125,12.3375,11.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16480,16480,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.985,12.16125,12.3375,11.43
16481,16481,"PEREIRA, VERONICA SHANTI",11.57,27.0,1,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.985,12.16125,12.3375,11.57
16483,16483,"HU, Chia-Chen",11.57,26.0,2,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.985,12.16125,12.3375,11.57
16553,16553,"HU, Chia-Chen",11.63,26.0,2,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.985,12.16125,12.3375,11.63


In [332]:
rslt_df = df.loc[(df['RESULT_CONV'] < df['35pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon'))]

In [333]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,Supanich Poolkerd,11.63,0.0,1,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.63
1,1,Kristina Knott,11.66,0.0,2,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.66
2,2,Zaidatul Husniah Zulkifli,11.69,0.0,3,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.69
3,3,Hoang Du Y,11.73,0.0,4,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.73
4,4,Elizabeth-Ann Tan Shee Ru,12.0,0.0,5,100m,Sprint,Female,SEA Games,None,0.0,11.75,11.75,11.9850,12.16125,12.3375,12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16572,16572,"TAN, ELIZABETH ANN SHEE R",12.04,20.0,3,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,12.04
16575,16575,"ALING, NUR AISHAH ROFINA",12.15,24.0,4,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,12.15
16581,16581,"TAN, ELIZABETH-ANN",11.99,21.0,1,100m,Sprint,Female,84th Singapore Open Track & Field,2024,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.99
16594,16594,"TAN, ELIZABETH-ANN",12.00,21.0,1,100m,Sprint,Female,84th Singapore Open Track & Field,2024,0.0,11.75,11.75,11.9850,12.16125,12.3375,12.00


In [334]:
rslt_df.tail(50)

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
15477,15477,"O'Neill, Mitchell",10.70,25.0,4,100m,Sprint,Male,83rd Singapore Open Track & Field,2023,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.70
15479,15479,"King, Leonard",10.72,26.0,5,100m,Sprint,Male,83rd Singapore Open Track & Field,2023,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.72
15502,15502,"LOUIS, MARC BRIAN",10.36,22.0,1,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.36
15505,15505,"WEI, TAI-SHENG",10.62,24.0,2,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.62
15506,15506,"S/O SUBASH SOMAN, PRAHARSH RYAN",10.74,26.0,3,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.74
15508,15508,"WEI, TAI-SHENG",10.69,24.0,2,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.69
15510,15510,"S/O SUBASH SOMAN, PRAHARSH RYAN",10.75,26.0,3,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.75
15515,15515,"ANG, JAMIE EL-REDHA",10.80,20.0,5,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.80
15521,15521,"LOUIS, MARC BRIAN",10.56,22.0,1,100m,Sprint,Male,84th Singapore Open Track & Field,2024,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.56
15562,15562,"Louis, Marc",10.64,21.0,1,100m,Sprint,Male,52nd SA Inter Club Championships 2023,2023,13.0,10.443,10.443,10.65186,10.808505,10.96515,10.64


In [335]:
rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
rslt_df=rslt_df.loc[rslt_df['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/170725308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']


In [336]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [337]:
rslt_df.sort_values(['EVENT', 'GENDER', 'Delta35'], ascending=[True, True, False], inplace=True)
rslt_df['overall_rank'] = 1
rslt_df['overall_rank'] = rslt_df.groupby(['EVENT'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/380210953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.sort_values(['EVENT', 'GENDER', 'Delta35'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/380210953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/380210953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [338]:
rslt_df.tail(50)

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta35,overall_rank
15690,15690,"Mamada, Kodai",21.50,22.0,2,200m,Sprint,Male,SA Allcomers Meet 3,2024,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.50,0.25570,17
15711,15711,"King, Leonard",21.50,26.0,4,200m,Sprint,Male,83rd Singapore Open Track & Field,2023,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.50,0.25570,18
15737,15737,"LEE, MARK REN",21.54,20.0,1,200m,Sprint,Male,84th Singapore Open Track & Field,2024,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.54,0.21570,19
15725,15725,"LEE, REUBEN RAINER",21.58,21.0,3,200m,Sprint,Male,83rd Singapore Open Track & Field,2023,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.58,0.17570,20
3014,3014,"Shimada, Kaishin",21.60,22.0,5,200m,Sprint,Male,SA Allcomers Meet 3,2024,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.60,0.15570,21
15729,15729,"King, Leonard",21.65,26.0,4,200m,Sprint,Male,83rd Singapore Open Track & Field,2023,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.65,0.10570,22
15733,15733,"S/O SUBASH SOMAN, PRAHARSH RYAN",21.67,26.0,2,200m,Sprint,Male,84th Singapore Open Track & Field,2024,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.67,0.08570,23
3015,3015,"Ryan, Praharsh",21.69,26.0,6,200m,Sprint,Male,SA Allcomers Meet 3,2024,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.69,0.06570,24
15732,15732,"LEE, MARK REN",21.69,19.0,7,200m,Sprint,Male,83rd Singapore Open Track & Field,2023,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.69,0.06570,25
15714,15714,"PengiranAidil, PengiranAidil",21.71,18.0,5,200m,Sprint,Male,83rd Singapore Open Track & Field,2023,14.0,21.02,21.02,21.4404,21.75570,22.0710,21.71,0.04570,26


In [339]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta35,overall_rank
16480,16480,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.43,0.73125,1
16481,16481,"PEREIRA, VERONICA SHANTI",11.57,27.0,1,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.57,0.59125,2
16483,16483,"HU, Chia-Chen",11.57,26.0,2,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.57,0.59125,3
16553,16553,"HU, Chia-Chen",11.63,26.0,2,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.63,0.53125,4
16559,16559,"TAN, ELIZABETH ANN SHEE R",11.91,20.0,3,100m,Sprint,Female,83rd Singapore Open Track & Field,2023,0.0,11.75,11.75,11.9850,12.16125,12.3375,11.91,0.25125,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15222,15222,"Jie Cong Jayden, Tan",1:57.22,18.0,1,800m,Mid,Male,84th Singapore Open Track & Field,2024,16.0,1:53.86,113.86,116.1372,117.84510,119.5530,117.22,0.62510,4
15215,15215,"Chia Wei En, Shawn",1:57.24,25.0,2,800m,Mid,Male,84th Singapore Open Track & Field,2024,16.0,1:53.86,113.86,116.1372,117.84510,119.5530,117.24,0.60510,5
15213,15213,"LIM, OLIVER",1:57.35,24.0,3,800m,Mid,Male,84th Singapore Open Track & Field,2024,16.0,1:53.86,113.86,116.1372,117.84510,119.5530,117.35,0.49510,6
15192,15192,"WAN ZAHARI, WAN MUHAMMAD FAZRI",1:57.39,22.0,1,800m,Mid,Male,83rd Singapore Open Track & Field,2023,16.0,1:53.86,113.86,116.1372,117.84510,119.5530,117.39,0.45510,7
